In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,GlobalAvgPool2D,Input
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator
from tensorflow.keras import callbacks,optimizers
import numpy as np
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/animal_classification/

/content/drive/MyDrive/animal_classification


In [ ]:
#!unzip archive.zip

In [ ]:
from translate import translate

In [ ]:
os.listdir('raw-img')

['cane',
 'horse',
 'elefante',
 'farfalla',
 'gallina',
 'gatto',
 'mucca',
 'sheep',
 'spider',
 'scoiattolo']

In [ ]:
for i in os.listdir('raw-img'):
  try:
    os.rename('raw-img/'+i, 'raw-img/'+translate[i])
  except Exception as e:
    print(e)

os.rename('raw-img/ragno','raw-img/spider')
#os.rename('raw-img/scoiattolo', 'raw-img/squirrel')


'horse'
'sheep'


In [ ]:
!ls raw-img/

butterfly  cat	chicken  cow  dog  elephant  horse  sheep  spider  squirrel


In [ ]:
for i in os.listdir('raw-img'):
  print(i, len(os.listdir('raw-img/' + i)))

dog 4863
horse 2623
elephant 1426
butterfly 2104
chicken 3098
cat 1668
cow 1866
sheep 1806
spider 4821
squirrel 1862


In [ ]:
try:
  os.mkdir("train") #1000 in train
  os.mkdir("test") #approx 400 in test
except:
  pass
for i in os.listdir('raw-img'):
  try:
    os.mkdir("train/"+i)
    os.mkdir("test/"+i)
  except:
    pass
  for j in os.listdir("raw-img/"+i)[:1000]:
    os.rename("raw-img/"+ i + "/" + j, "train/"+ i + "/" +j)
  for j in os.listdir("raw-img/"+i)[:400]:
    os.rename("raw-img/"+ i + "/" + j, "test/"+ i + "/" +j)


In [ ]:
def img_Data(dir_path, target_size, batch, class_lst, preprocessing):
  if preprocessing:
    gen_object = ImageDataGenerator(preprocessing_function=preprocessing)
  else:
    gen_object = ImageDataGenerator()

  return (gen_object.flow_from_directory(dir_path, 
                                        target_size = target_size,
                                        batch_size = batch, 
                                        class_mode='sparse', 
                                        classes = class_lst, 
                                        shuffle=True))

In [ ]:
train_data_gen = img_Data("train", (224, 224), 500, os.listdir("train"), preprocess_input)
valid_data_gen = img_Data("test", (224, 224), 500, os.listdir("test"), preprocess_input)

Found 9540 images belonging to 10 classes.
Found 3861 images belonging to 10 classes.


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    alpha=1.0,
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
elst = callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')
save_ck = callbacks.ModelCheckpoint('.mdl_wt.hdf5', save_best_only=True, monitor='val_loss', mode='min')


In [ ]:
model.fit(train_data_gen, batch_size=500, validation_data = valid_data_gen, callbacks=[elst, save_ck], epochs=10)

Epoch 1/10
20/20 [==============================] - 863s 43s/step - loss: 0.3915 - accuracy: 0.8736 - val_loss: 0.2016 - val_accuracy: 0.9422
Epoch 2/10
20/20 [==============================] - 835s 42s/step - loss: 0.1352 - accuracy: 0.9604 - val_loss: 0.2113 - val_accuracy: 0.9417
Epoch 3/10
20/20 [==============================] - 807s 41s/step - loss: 0.1024 - accuracy: 0.9700 - val_loss: 0.1873 - val_accuracy: 0.9482
Epoch 4/10
20/20 [==============================] - 840s 43s/step - loss: 0.0711 - accuracy: 0.9789 - val_loss: 0.1740 - val_accuracy: 0.9498
Epoch 5/10
20/20 [==============================] - 834s 42s/step - loss: 0.0469 - accuracy: 0.9878 - val_loss: 0.1828 - val_accuracy: 0.9469
Epoch 6/10
20/20 [==============================] - 837s 42s/step - loss: 0.0357 - accuracy: 0.9909 - val_loss: 0.1922 - val_accuracy: 0.9459
Epoch 7/10
20/20 [==============================] - 812s 41s/step - loss: 0.0244 - accuracy: 0.9959 - val_loss: 0.1807 - val_accuracy: 0.9526
Epoch 